# Compare Tycho and Aurora

In [ ]:
# %load imports.py
# %load ../imports.py
%matplotlib inline
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import matplotlib.pyplot as pltb
import matplotlib.pyplot as plt
import seaborn as sns
width=20
height=3
plt.rcParams["figure.figsize"] = (width,height)
sns.set(rc={'figure.figsize':(width,height)})

#import seaborn as sns
import os
from collections import OrderedDict

from IPython.display import display

pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option("display.max_columns", None)

import folium
import plotly.express as px
import plotly.graph_objects as go

import sys
import os

from sklearn.metrics import r2_score
import seaborn as sns

import statsmodels.api as sm

from d2e2f.visualization import visualize
import re
import plotly.graph_objects as go

In [ ]:
#loaded = catalog.load('trip_statistics_joined_thrusters')
#df = pd.DataFrame()
#for partition_id, loader in loaded.items():
#    df_ = loader()
#    df_['partition_id'] = partition_id
#    df_['start_time'] = pd.to_datetime(df_['start_time'], utc=True)
#    df_['end_time'] = pd.to_datetime(df_['end_time'], utc=True)
#    
#    df = df.append(df_, ignore_index=True)
#
#df.sort_values(by='start_time', inplace=True)

In [ ]:
ship_names = ['tycho','aurora']

df = pd.DataFrame()
for ship_name in ship_names:
    df_ = catalog.load(f'{ship_name}.trip_statistics_joined_thrusters')
    df_['ship'] = ship_name
    df_['start_time'] = pd.to_datetime(df_['start_time'], utc=True)
    df_['end_time'] = pd.to_datetime(df_['end_time'], utc=True)
    
    df = df.append(df_, ignore_index=True)

df.sort_values(by='start_time', inplace=True)

In [ ]:
df.head()

In [ ]:
df['ship'].unique()

In [ ]:
#df['ship'] = df['partition_id'].apply(lambda x:re.search(r'([^-]+)',x).groups(1)[0])

In [ ]:
fig = px.scatter(df, x='start_time',y='P', color='ship', width=1000, height=600, 
                color_discrete_sequence=['red','green'], hover_data=['trip_no','trip_time'])
fig.show()

In [ ]:
ships = df.groupby(by='ship')

In [ ]:
df_tycho=ships.get_group('tycho')
df_aurora=ships.get_group('aurora')


In [ ]:
df_tycho.info()

In [ ]:
df_tycho.head()

In [ ]:
str(df_tycho['start_time'].iloc[0])

In [ ]:
(df_tycho['start_time'].diff().iloc[1:] > "0").all()

In [ ]:
mask = df_tycho['start_time'].diff() > "1 days"

In [ ]:
mask = df_aurora['start_time'].apply(lambda x: ((x-df_tycho['start_time']).abs() < "0 days 01:00:00").any())
df_aurora = df_aurora.loc[mask].copy()

In [ ]:
df_cut = pd.concat([df_tycho, df_aurora])
df_cut.sort_values(by='start_time', inplace=True)
fig = px.scatter(df_cut, x='start_time',y='P', color='ship', width=1000, height=600, 
                color_discrete_sequence=['red','green'], hover_data=['trip_no','trip_time'])
fig.show()

In [ ]:
fig = px.line(df_cut, x='start_time',y='w', color='ship', width=1000, height=600, 
                color_discrete_sequence=['red','green'], hover_data=['trip_no','trip_time'])
fig.show()

In [ ]:
sns.displot(df_cut, x='P', hue='ship', kind="kde", bw_adjust=1, aspect=3)

In [ ]:
df_tycho.describe()

In [ ]:
df_aurora.describe()

## Following trips

In [ ]:
for trip_direction, df_aurora_ in df_aurora.groupby(by='trip_direction'):
    df_tycho_ = df_tycho.groupby(by='trip_direction').get_group(trip_direction)
    
    close_indexes = df_aurora_['start_time'].apply(lambda x: ((x-df_tycho_['start_time']).abs()).idxmin())
    
    #df['P_aurora'] = df_aurora_['P']
    #df['P_tycho'] = df_tycho_.loc[close_indexes]['P'].values
    
    df_aurora_.reset_index(inplace=True)
    df_tycho_ = df_tycho_.loc[close_indexes].reset_index()
    
    df = pd.merge(df_aurora_, df_tycho_, how='inner', left_index=True, right_index=True, suffixes = ('_aurora', '_tycho'))
    
    
    df['P_min'] = df[['P_aurora','P_tycho']].min(axis=1)
    df['P_max'] = df[['P_aurora','P_tycho']].max(axis=1)
    df['P_tot'] = df[['P_aurora','P_tycho']].sum(axis=1)
    
    df['P_aurora - P_tycho'] = df['P_aurora'] - df['P_tycho']
    df['sog_aurora - sog_tycho'] = df['sog_aurora'] - df['sog_tycho']
    
    energy_saving = (df['P_max'] - df['P_min']).sum() / df['P_tot'].sum()
    
    fig = px.scatter(df, x='P_aurora',y='P_tycho', width=1000, height=600, 
                     opacity=0.3, 
                     hover_data=['P_aurora','P_tycho','sog_aurora', 'sog_tycho'],
                     title=f'{trip_direction} energy saving: {np.round(energy_saving*100)}%')
    fig.show()
    
    
    fig = px.scatter(df, x='sog_aurora - sog_tycho',y='P_aurora - P_tycho', width=1000, height=600, 
                     opacity=0.3, 
                     hover_data=['P_aurora','P_tycho','sog_aurora', 'sog_tycho'],
                     title=f'{trip_direction} energy saving: {np.round(energy_saving*100)}%')
    fig.show()
    

In [ ]:
for trip_direction, df_aurora_ in df_aurora.groupby(by='trip_direction'):
    df_tycho_ = df_tycho.groupby(by='trip_direction').get_group(trip_direction)
    
    df = pd.DataFrame()
    df['w_aurora'] = df_aurora_['w']
    df['start_aurora'] = df_aurora_['start_time']
    close_indexes = df_aurora_['start_time'].apply(lambda x: ((x-df_tycho_['start_time']).abs()).idxmin())
    df['w_tycho'] = df_tycho_.loc[close_indexes]['w'].values
    df['start_tycho'] = df_tycho_.loc[close_indexes]['start_time'].values

    fig = px.scatter(df, x='w_aurora',y='w_tycho', width=1000, height=600, 
                     opacity=0.3, title=f'{trip_direction}')
            
    fig.show()
    
    #fig = px.scatter(df, x='start_aurora',y='start_tycho', width=1000, height=600, 
    #                 opacity=0.3, title=f'{trip_direction}')
    #fig.show()

In [ ]:
fig = px.scatter_3d(df_cut, x='start_time', y='sog', z='P', color='ship',
              opacity=0.5,
                    width=1000,
                    height=800
    )

fig.update_traces(marker=dict(size=2,
                         ))

fig.show()

In [ ]:
def select_features(df):
    
    data = df[['P','sog']].copy()
    data.sort_values(by='sog', inplace=True)
    data.set_index('sog', inplace=True)
    y = data.pop('P')
    X = data
    X['sog**3'] = X.index**3
    
    return y,X

def fit_sog3(df):
    
    y,X = select_features(df)
    model = sm.OLS(y,X)
    results = model.fit()
    
    return results

In [ ]:
models = ships.apply(fit_sog3)
models

In [ ]:
ships = df_cut.groupby(by='ship', sort=False)
P_pred = pd.Series(dtype=float)
for ship_name, ship in ships:
    model = models[ship_name]
    
    P_pred = P_pred.append(model.predict(ship[['sog']]**3))

df_cut['P_pred'] = P_pred

In [ ]:
fig = px.scatter(df_cut, x='sog',y='P', color='ship', width=1000, height=600, 
                color_discrete_sequence=['red','green'], hover_data=['trip_no','trip_time'], opacity=0.2)

for ship_name, model in models.items():
    
    sog = np.linspace(df_cut['sog'].min(), df_cut['sog'].max(), 100)
    df_ = pd.DataFrame()
    df_['sog**3'] = sog**3
    P_pred = model.predict(df_)
    
    line = go.Scatter(x=sog, y=P_pred, name=ship_name)

    fig.add_trace(line)

fig.show()

In [ ]:
df_cut['P_rest'] = df_cut['P'] - df_cut['P_pred']

In [ ]:
df_cut['trip_direction'].unique()

In [ ]:
trip_direction='Helsingborg-Helsingør'
df_ = df_cut.groupby(by='trip_direction').get_group(trip_direction)

fig = px.scatter(df_, x='start_time',y='P_rest', color='ship', width=1000, height=600, 
            color_discrete_sequence=['red','green'], hover_data=['trip_no','trip_time'], title=trip_direction)
fig.show()

fig = px.scatter(df_, x='start_time',y='w', color='ship', width=1000, height=600, 
            color_discrete_sequence=['red','green'], hover_data=['trip_no','trip_time'], title=trip_direction)
fig.show()

fig = px.scatter(df_, x='start_time',y='aw', color='ship', width=1000, height=600, 
            color_discrete_sequence=['red','green'], hover_data=['trip_no','trip_time'], title=trip_direction)
fig.show()

fig = px.scatter(df_, x='start_time',y='aw_x', color='ship', width=1000, height=600, 
            color_discrete_sequence=['red','green'], hover_data=['trip_no','trip_time'], title=trip_direction)
fig.show()

fig = px.scatter(df_, x='start_time',y='aw_y', color='ship', width=1000, height=600, 
            color_discrete_sequence=['red','green'], hover_data=['trip_no','trip_time'], title=trip_direction)
fig.show()

In [ ]:
for trip_direction,df_ in df_cut.groupby(by='trip_direction'):
    fig = px.scatter(df_, x='start_time',y='P_rest', color='ship', width=1000, height=600, 
                color_discrete_sequence=['red','green'], hover_data=['trip_no','trip_time'], title=trip_direction)
    fig.show()
    
    fig = px.scatter(df_, x='start_time',y='w', color='ship', width=1000, height=600, 
                color_discrete_sequence=['red','green'], hover_data=['trip_no','trip_time'], title=trip_direction)
    fig.show()
    
    fig = px.scatter(df_, x='start_time',y='aw_x', color='ship', width=1000, height=600, 
                color_discrete_sequence=['red','green'], hover_data=['trip_no','trip_time'], title=trip_direction)
    fig.show()

In [ ]:
for trip_direction,df_ in df_cut.groupby(by='trip_direction'):
    fig = px.scatter_3d(df_, x='start_time', y='w', z='P_rest', color='ship',
                  opacity=0.5,
                        width=1000,
                        height=800, title=trip_direction
        )
    
    fig.update_traces(marker=dict(size=2,
                             ))
    
    fig.show()